In [1]:
import os 
import duckdb 
from dotenv import load_dotenv

In [2]:
# loading the env 
load_dotenv()

# setting up the paths 
base_path = os.getenv('base_path')
database = os.getenv('database_path')

In [3]:
# setting up the in memory database 
database_path = f"{database}/database.db"

# connecting to database file 
ddb = duckdb.connect(database_path)

In [4]:
# create table dim_seller and create all of its relationships 
ddb.execute(
    '''
        CREATE TABLE IF NOT EXISTS dim_sellers AS 
        SELECT 
            s.*, 
            g.GeoLocId
        FROM
            read_csv_auto(?) s 
        JOIN 
            dim_geolocation g 
        ON
            s.seller_zip_code_prefix = g.GeoLocZipCodePrefix
        AND 
            s.seller_city = g.GeoLocCity
        AND 
            s.seller_state = g.GeoLocState;
    ''',
    (f"{base_path}/datasets/olist_sellers_dataset.csv",)
)

In [5]:
# renaming tables in dim_seller 
ddb.sql(
    '''
        ALTER TABLE dim_sellers 
        RENAME seller_id TO SellerId;
    '''
)

ddb.sql(
    '''
        ALTER TABLE dim_sellers 
        RENAME seller_zip_code_prefix TO SellerZipCodePrefix;
    '''
)

ddb.sql(
    '''
        ALTER TABLE dim_sellers 
        RENAME seller_city TO SellerCity;
    '''
)

ddb.sql(
    '''
        ALTER TABLE dim_sellers 
        RENAME seller_state TO SellerState;
    '''
)

In [6]:
ddb.sql(
    '''
        SELECT *
        FROM dim_sellers;
    '''
)

┌──────────────────────────────────┬─────────────────────┬────────────┬─────────────┬──────────────────────────────────────┐
│             SellerId             │ SellerZipCodePrefix │ SellerCity │ SellerState │               GeoLocId               │
│             varchar              │       varchar       │  varchar   │   varchar   │                 uuid                 │
├──────────────────────────────────┼─────────────────────┼────────────┼─────────────┼──────────────────────────────────────┤
│ ad420dd0c4f92f8af951ac24b86d0cf5 │ 38230               │ fronteira  │ MG          │ c0d6d646-676e-4e2c-8e17-3fa80b95c9b5 │
│ ad420dd0c4f92f8af951ac24b86d0cf5 │ 38230               │ fronteira  │ MG          │ 6b540dd8-5332-489e-bcc7-9a2a311c672e │
│ ad420dd0c4f92f8af951ac24b86d0cf5 │ 38230               │ fronteira  │ MG          │ f2c8cbdc-4b88-489e-84d7-c5d73a121cf6 │
│ ad420dd0c4f92f8af951ac24b86d0cf5 │ 38230               │ fronteira  │ MG          │ 66641bdd-897a-4887-881c-2df903e63eaf │


In [7]:
# Checking the data types of every column
ddb.sql(
    '''
        SELECT column_name, data_type
        FROM information_schema.columns
        WHERE table_name = 'dim_sellers';
    '''
)

┌─────────────────────┬───────────┐
│     column_name     │ data_type │
│       varchar       │  varchar  │
├─────────────────────┼───────────┤
│ SellerId            │ VARCHAR   │
│ SellerZipCodePrefix │ VARCHAR   │
│ SellerCity          │ VARCHAR   │
│ SellerState         │ VARCHAR   │
│ GeoLocId            │ UUID      │
└─────────────────────┴───────────┘

In [8]:
# Checking null values in dim_sellers table 
ddb.sql(
    '''
        SELECT
            COUNT(*) AS total_rows,
            COUNT(CASE WHEN SellerId IS NULL THEN 1 END) AS SellerId,
            COUNT(CASE WHEN SellerZipCodePrefix IS NULL THEN 1 END) AS SellerZipCodePrefix,
            COUNT(CASE WHEN SellerCity IS NULL THEN 1 END) AS SellerCity,
            COUNT(CASE WHEN SellerState IS NULL THEN 1 END) AS SellerState,
            COUNT(CASE WHEN GeoLocId IS NULL THEN 1 END) AS GeoLocId,
        FROM 
            dim_sellers;
    '''
)

┌────────────┬──────────┬─────────────────────┬────────────┬─────────────┬──────────┐
│ total_rows │ SellerId │ SellerZipCodePrefix │ SellerCity │ SellerState │ GeoLocId │
│   int64    │  int64   │        int64        │   int64    │    int64    │  int64   │
├────────────┼──────────┼─────────────────────┼────────────┼─────────────┼──────────┤
│     385584 │        0 │                   0 │          0 │           0 │        0 │
└────────────┴──────────┴─────────────────────┴────────────┴─────────────┴──────────┘

In [9]:
# Checking and changing the datatypes of the dim_customers table to its appropriate data 
# 1. Add a new column with the correct type
ddb.sql('''
    ALTER TABLE dim_sellers
    ALTER COLUMN SellerZipCodePrefix SET DATA TYPE INT;
''')

In [10]:
# Define the output directory and ensure it exists
output_dir = "/Users/macintoshcider/Documents/Programming/Python/ETL/climate-commerce/analytics"
output_file = os.path.join(output_dir, "dim_sellers.csv")
ddb.execute(
    f"""
        COPY dim_sellers TO '{output_file}' (FORMAT CSV, HEADER);
    """
)
print(f"Data successfully exported to {output_file}")

Data successfully exported to /Users/macintoshcider/Documents/Programming/Python/ETL/climate-commerce/analytics/dim_sellers.csv


In [11]:
# finally closing the duckdb connection
ddb.close()